<h2><a href="https://leetcode.com/problems/highest-grade-for-each-student">1214. Highest Grade For Each Student</a></h2><h3>Medium</h3><hr><p>Table: <code>Enrollments</code></p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| course_id     | int     |
| grade         | int     |
+---------------+---------+
(student_id, course_id) is the primary key (combination of columns with unique values) of this table.
grade is never NULL.
</pre>

<p>&nbsp;</p>

<p>Write a solution to find the highest grade with its corresponding course for each student. In case of a tie, you should find the course with the smallest <code>course_id</code>.</p>

<p>Return the result table ordered by <code>student_id</code> in <strong>ascending order</strong>.</p>

<p>The&nbsp;result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Enrollments table:
+------------+-------------------+
| student_id | course_id | grade |
+------------+-----------+-------+
| 2          | 2         | 95    |
| 2          | 3         | 95    |
| 1          | 1         | 90    |
| 1          | 2         | 99    |
| 3          | 1         | 80    |
| 3          | 2         | 75    |
| 3          | 3         | 82    |
+------------+-----------+-------+
<strong>Output:</strong> 
+------------+-------------------+
| student_id | course_id | grade |
+------------+-----------+-------+
| 1          | 2         | 99    |
| 2          | 2         | 95    |
| 3          | 3         | 82    |
+------------+-----------+-------+
</pre>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, row_number
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.appName("HighestGradePerStudent").getOrCreate()

# Hardcoded DataFrame
data = [
    (2, 2, 95),
    (2, 3, 95),
    (1, 1, 90),
    (1, 2, 99),
    (3, 1, 80),
    (3, 2, 75),
    (3, 3, 82)
]

columns = ["student_id", "course_id", "grade"]

df = spark.createDataFrame(data, columns)

25/02/19 08:32:54 WARN Utils: Your hostname, Nahils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
25/02/19 08:32:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/19 08:32:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.createOrReplaceTempView('Enrollments')

In [6]:
spark.sql("""
    SELECT *
    FROM      
        (SELECT *,
          RANK() OVER (PARTITION BY student_id ORDER BY grade DESC, course_id ASC) as grade_rn
          FROM Enrollments
        ) Temp
    WHERE grade_rn = 1  
          
""").show()

+----------+---------+-----+--------+
|student_id|course_id|grade|grade_rn|
+----------+---------+-----+--------+
|         1|        2|   99|       1|
|         2|        2|   95|       1|
|         3|        3|   82|       1|
+----------+---------+-----+--------+



In [7]:
spark.stop()